<a href="https://colab.research.google.com/github/wfsilva-uea/northwind/blob/master/northwind.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Instalar e configurar PostgreSQL

In [ ]:
# Install postgresql server
!sudo apt-get -y -qq update
!sudo apt-get -y -qq install postgresql
!sudo service postgresql start

In [ ]:
# Setup a password '123456' for username 'postgres'
!sudo -u postgres psql -U postgres -c "ALTER USER postgres PASSWORD '123456';"
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"

In [1]:
# Setup a database with name 'northwind' to be used
!sudo -u postgres psql -U postgres -c 'DROP DATABASE IF EXISTS northwind;'
!sudo -u postgres psql -U postgres -c 'CREATE DATABASE northwind;'

DROP DATABASE
CREATE DATABASE


### Popular a base de dados do PostgreSQL no Heroku

In [ ]:
# install postgres binary lib
!pip install psycopg2-binary

# getting sql file to runtime
!npx degit wfsilva-uea/northwind/northwind.sql northwind.sql -f

In [2]:
# Creating connection configuration
DB_HOST='localhost'
DB_PORT=5432
DB_USER='postgres'
DB_PASSWORD='123456'
DB_NAME='northwind'

%env DB_HOST=DB_HOST
%env DB_PORT=DB_PORT
%env DB_USER=DB_USER
%env DB_PASSWORD=DB_PASSWORD
%env DB_NAME=DB_NAME

env: DB_HOST=DB_HOST
env: DB_PORT=DB_PORT
env: DB_USER=DB_USER
env: DB_PASSWORD=DB_PASSWORD
env: DB_NAME=DB_NAME


In [3]:
!PGPASSWORD=$DB_PASSWORD psql -q -h $DB_HOST -p $DB_PORT -U $DB_USER -d $DB_NAME -f 'northwind.sql'

### Consultas

In [4]:
# requirements
import pandas as pd
from sqlalchemy import create_engine

In [9]:
# create alchemy url connection from heroku
ALCHEMY_CONNECTION = 'postgresql+psycopg2://{user}:{password}@{host}:{port}/{database}'.format(
  host=DB_HOST,
  port=DB_PORT,
  user=DB_USER,
  password=DB_PASSWORD,
  database=DB_NAME
)

# create sql alchemy engine
engine = create_engine(ALCHEMY_CONNECTION, echo=False)

In [10]:
# 1. os 10 produtos mais caros
query = """
  select distinct 
    products.product_name as ten_most_expensive_products, 
    products.unit_price 
  from products 
  order by products.unit_price desc
"""
df = pd.read_sql_query(query, con=engine)
df.head(10)

,ten_most_expensive_products,unit_price
0,Côte de Blaye,263.50
1,Thüringer Rostbratwurst,123.79
2,Mishi Kobe Niku,97.00
3,Sir Rodney's Marmalade,81.00
4,Carnarvon Tigers,62.50
5,Raclette Courdavault,55.00
6,Manjimup Dried Apples,53.00
7,Tarte au sucre,49.30
8,Ipoh Coffee,46.00
9,Rössle Sauerkraut,45.60


In [11]:
# 2. pedido trimestral
query = """
  select 
    customers.customer_id, 
    customers.company_name, 
    customers.city, 
    customers.country
  from customers 
  right join orders on customers.customer_id = orders.customer_id
  where orders.order_date between '1996-01-01' and '1996-12-31'
"""
df = pd.read_sql_query(query, con=engine)
df

,customer_id,company_name,city,country
0,VINET,Vins et alcools Chevalier,Reims,France
1,TOMSP,Toms Spezialitäten,Münster,Germany
2,HANAR,Hanari Carnes,Rio de Janeiro,Brazil
3,VICTE,Victuailles en stock,Lyon,France
4,SUPRD,Suprêmes délices,Charleroi,Belgium
...,...,...,...,...
147,HILAA,HILARION-Abastos,San Cristóbal,Venezuela
148,FRANK,Frankenversand,München,Germany
149,PRINI,Princesa Isabel Vinhos,Lisboa,Portugal
150,SAVEA,Save-a-lot Markets,Boise,USA


In [12]:
# 3. Detalhes do pedido estendido
query = """
  select distinct
    od.order_id, 
    od.product_id, 
    p.product_name, 
    od.unit_price, 
    od.quantity, 
    od.discount, 
    (od.unit_price * od.quantity * (1 - od.discount) / 100) * 100 as extended_price
  from products p
  inner join order_details od on p.product_id = od.product_id
  order by 
    od.order_id
"""
df = pd.read_sql_query(query, con=engine)
df

,order_id,product_id,product_name,unit_price,quantity,discount,extended_price
0,10248,11,Queso Cabrales,14.00,12,0.00,168.000000
1,10248,42,Singaporean Hokkien Fried Mee,9.80,10,0.00,98.000002
2,10248,72,Mozzarella di Giovanni,34.80,5,0.00,173.999996
3,10249,14,Tofu,18.60,9,0.00,167.400003
4,10249,51,Manjimup Dried Apples,42.40,40,0.00,1696.000061
...,...,...,...,...,...,...,...
2150,11077,64,Wimmers gute Semmelknödel,33.25,2,0.03,64.505000
2151,11077,66,Louisiana Hot Spiced Okra,17.00,1,0.00,17.000000
2152,11077,73,Röd Kaviar,15.00,2,0.01,29.700000
2153,11077,75,Rhönbräu Klosterbier,7.75,4,0.00,31.000000


In [13]:
# 4. Produtos por baixo do valor médio
query = """
  select distinct 
    p.product_name, 
    p.unit_price
  from products p
  where p.unit_price > (select avg(products.unit_price) from products)
  order by p.unit_price desc
"""
df = pd.read_sql_query(query, con=engine)
df

,product_name,unit_price
0,Côte de Blaye,263.50
1,Thüringer Rostbratwurst,123.79
2,Mishi Kobe Niku,97.00
3,Sir Rodney's Marmalade,81.00
4,Carnarvon Tigers,62.50
5,Raclette Courdavault,55.00
6,Manjimup Dried Apples,53.00
7,Tarte au sucre,49.30
8,Ipoh Coffee,46.00
9,Rössle Sauerkraut,45.60


In [14]:
# 5. Subtotal do pedido
query = """
  select distinct 
    od.order_id, 
    sum((od.unit_price * od.quantity * (1 - od.discount) / 100) * 100) as subtotal
  from order_details od
  group by od.order_id
"""
df = pd.read_sql_query(query, con=engine)
df

,order_id,subtotal
0,10605,4109.700019
1,10744,735.999982
2,10312,1614.800001
3,10765,1515.599943
4,10833,906.929992
...,...,...
825,10279,351.000009
826,10673,412.350002
827,10662,125.000000
828,10283,1414.800020


In [15]:
# 6. Vendas por ano
query = """
  select distinct 
    extract(year from o.order_date) :: varchar(4) as year, 
    sum((od.unit_price * od.quantity * (1 - od.discount) / 100) * 100) as subtotal
  from orders o 
  inner join order_details od on o.order_id = od.order_id
  group by year
  order by year desc
"""
df = pd.read_sql_query(query, con=engine)
df

,year,subtotal
0,1998,440623.865278
1,1997,617085.202393
2,1996,208083.970983
